In [6]:
# Imports
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from numpy import asarray
from numpy import savetxt
from scipy.linalg import svd
from sklearn.mixture import BayesianGaussianMixture
from sklearn.preprocessing import StandardScaler, PowerTransformer

In [7]:
# Read the data
mirna_data = pd.read_csv(r'MDICC-data\BRCA\original_data\mirna.csv')
gene_data = pd.read_csv(r'MDICC-data\BRCA\original_data\gene.csv')
methy_data = pd.read_csv(r'MDICC-data\BRCA\original_data\methyl.csv')

In [8]:
# Remove the first column of each dataset (the feature names)
mirna_data.pop(mirna_data.columns[0])
gene_data.pop(gene_data.columns[0])
methy_data.pop(methy_data.columns[0])

0        cg00000292
1        cg00002426
2        cg00003994
3        cg00005847
4        cg00007981
            ...    
22528    cg27657249
22529    cg27661264
22530    cg27662379
22531    cg27662877
22532    cg27665659
Name: Unnamed: 0, Length: 22533, dtype: object

In [9]:
def scaling(values):
    # Scale Z-score
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(values.T)

    # Power Transformation
    transformer = PowerTransformer(method='yeo-johnson', standardize=False)
    data_transformed = transformer.fit_transform(scaled_data)
    
    return data_transformed

In [10]:
def get_data(datasets):
    data = []
    dataset_names = ['miRNA', 'gene expression', 'methylation']
   
    for x in datasets:
        # Preprocess the data
        X = x.to_numpy()
        
        # Calculate the percentage of zeros in each row
        zero_percentage = np.mean(X == 0, axis=1)

        # Get the indices of the rows with less than 20% zeros
        keep_indices = np.where(zero_percentage < 0.2)[0]

        # Remove the rows with 20% or more zeros
        X_without_zeros = X[keep_indices, :]
        
        # Change 0 values to NaN for the KNN imputer
        X_without_zeros[X_without_zeros == 0] = np.nan
        
        # Determine the amount of neighbours
        N = round(np.sqrt(x.shape[1]))
        
        # Start KNN imputer
        imputer = KNNImputer(n_neighbors=N+2)
        X = imputer.fit_transform(X_without_zeros)
 
        # Scale the data without the missing values
        dataset = scaling(X)
        
        # Perform Bayesian Gaussian Mixture model with the diagonal covariance matrix
        bgmm_model = BayesianGaussianMixture(n_components = dataset.shape[0], covariance_type='diag')
        bgmm_model.fit(dataset)
        
        # Get covariances
        covariances = bgmm_model.covariances_

        # Convert covariances to a NumPy array
        covariances = np.array(covariances)

        # Calculate the variances for each feature across components
        variances = np.var(covariances, axis=0)

        # Sort the variances in descending order
        sorted_indices = np.argsort(variances)[::-1]
        sorted_variances = variances[sorted_indices]

        # Calculate the cumulative explained variance
        cumulative_variance = np.cumsum(sorted_variances) / np.sum(sorted_variances)

        # Determine the optimal number of features based on a threshold for cumulative variance explained
        threshold = 0.95
        num_features = np.sum(cumulative_variance < threshold) + 1
        
        # Save the selected features in the variable
        selected_features = dataset[:, sorted_indices[:num_features]]

        # Add the new dataset to data
        data.append(selected_features)
        
    return data

In [11]:
def calc_relation(X, Y):


    # Calculate Canonical Correlations by performaing Singular Value Decomposition (SVD)
    U, D, V = svd(X.T @ Y)  
    
    # Get the rank of D (the number of non-zero canonical correlations)
    r = np.linalg.matrix_rank(D)

    # Select Significant Canonical Correlations
    
    # Determine the number of canonical correlations to keep
    n = min(X.shape[1], Y.shape[1]) 
    
    # Select r significant canonical correlatoins
    canonical_correlations = D[:r]  

    # Get canonical variates for predictor dataset
    Wx = X @ U[:, :r]  
    
    # Get canonical variates for response dataset
    Wy = Y @ V[:, :r]  
   
    # Compute distance matrix based on the canonical variates
    num_samples = len(Wx)
    
    distances = np.zeros((num_samples, num_samples))
    
    for i in range(num_samples):
        for j in range(i+1, num_samples): 
            # Compute the Euclidean distance between canonical variates
            distance = np.sqrt((Wx[i]-Wx[j])**2 + (Wy[i] - Wy[j])**2)
            distances[i, j] = distance
            distances[j, i] = distance  
    
    
    return distances

In [12]:
# parameter setting
k1 = 18 # the neighbor of affinity matrix
k2 = 42 # 
k3 = 2  # number of cluster
c  = 3  # c = k3(c>2) or c = 3(c=2)

In [13]:
# Creating the affinity matrices in the same way as the MDICC paper did,
# I would like to refer to the original paper:
# Yang, Y., Tian, S., Qiu, Y., Zhao, P., & Zou, Q. (2022). 
# MDICC: Novel method for multi-omics data integration and cancer 
# subtype identification. Briefings in Bioinformatics, 23(3), bbac132.

def kscale(matrix, k=7, dists=None, minval=0.004):
    """ Returns the local scale based on the k-th nearest neighbour """
    r,c = matrix.shape
    scale = np.zeros((r,c))
    for i in range(1,(k+1)):
        ix = (np.arange(len(matrix)), matrix.argsort(axis=0)[i])
        d = matrix[ix][np.newaxis].T
        dists = (d if dists is None else dists)
        scale1 = dists.dot(dists.T)
        
        scale = scale1 + scale
    scale = scale/k
    return np.clip(scale, minval, np.inf)


def affinity(matrix, k):
    scale = kscale(matrix, k)
    msq = matrix * matrix
    scaled = -msq /(0.5*scale+0.5*matrix)
    scaled[np.where(np.isnan(scaled))] = 0.0
    a = np.exp(scaled)
    a.flat[::matrix.shape[0]+1] = 0.0  # zero out the diagonal
    return a


def testaff(matrix,k):
    k = int(k)
    a = matrix
    affi = affinity(a,k)
    return affi

In [14]:
datasets = [mirna_data, gene_data, methy_data]
data = get_data(datasets)

C:\Users\markp\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


KeyboardInterrupt: 

In [11]:
# Compute the affinity matrix for gene expression as predictor dataset and miRNA as response dataset
d10 = calc_relation(data[1], data[0])
A10 = testaff(d10, k1)
pd.DataFrame(asarray(A10)).to_csv("A10_BRCA.csv")

# Compute the affinity matrix for miRNA as predictor dataset and gene epxression as response dataset
d01 = calc_relation(data[0], data[1])
A01 = testaff(d01, k1)
pd.DataFrame(asarray(A01)).to_csv("A01_BRCA.csv")

# Compute the affinity matrix for gene expression as predictor dataset and methylation as response dataset
d12 = calc_relation(data[1], data[2])
A12 = testaff(d12, k1)
pd.DataFrame(asarray(A12)).to_csv("A12_BRCA.csv")

# Compute the affinity matrix for methylation as predictor dataset and gene epxression as response datas
d21 = calc_relation(data[2], data[1])
A21 = testaff(d21, k1)
pd.DataFrame(asarray(A21)).to_csv("A21_BRCA.csv")

# Compute the affinity matrix for methylation as predictor dataset and miRNA as response datas
d20 = calc_relation(data[2], data[0])
A20 = testaff(d20, k1)
pd.DataFrame(asarray(A20)).to_csv("A20_BRCA.csv")

# Compute the affinity matrix for miRNA as predictor dataset and methylation as response datas
d02 = calc_relation(data[0], data[2])
A02 = testaff(d02, k1)
pd.DataFrame(asarray(A02)).to_csv("A02_BRCA.csv")
